<a href="https://colab.research.google.com/github/savarirohan3-byte/AIDS-Learning/blob/main/SPL_YES_OR_NO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# =========================
# 1. INSTALL & IMPORT
# =========================
!pip install tensorflow pandas numpy

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate
from sklearn.model_selection import train_test_split

In [11]:
data = pd.read_csv("Top Indian Places to Visit.csv")
data.head()

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
0,0,Northern,Delhi,Delhi,India Gate,War Memorial,1921,0.5,4.6,0,Yes,NaN,Historical,Yes,2.60,Evening
1,1,Northern,Delhi,Delhi,Humayun's Tomb,Tomb,1572,2.0,4.5,30,Yes,NaN,Historical,Yes,0.40,Afternoon
2,2,Northern,Delhi,Delhi,Akshardham Temple,Temple,2005,5.0,4.6,60,Yes,NaN,Religious,No,0.40,Afternoon
3,3,Northern,Delhi,Delhi,Waste to Wonder Park,Theme Park,2019,2.0,4.1,50,Yes,Monday,Environmental,Yes,0.27,Evening
4,4,Northern,Delhi,Delhi,Jantar Mantar,Observatory,1724,2.0,4.2,15,Yes,NaN,Scientific,Yes,0.31,Morning


In [12]:
stories = []
questions = []
answers = []

for i in range(len(data)):

    place = str(data.loc[i, "Name"])
    state = str(data.loc[i, "State"])
    city = str(data.loc[i, "City"])
    typ = str(data.loc[i, "Type"])
    best_time = str(data.loc[i, "Best Time to visit"])
    rating = str(data.loc[i, "Google review rating"])

    # STORY = all information about that place
    story = f"{place} is in {city} {state}. It is a {typ}. Best time to visit is {best_time}. Rating is {rating}."

    # QA PAIRS

    questions.append(f"Where is {place} located")
    answers.append(state)
    stories.append(story)

    questions.append(f"What type of place is {place}")
    answers.append(typ)
    stories.append(story)

    questions.append(f"Best time to visit {place}")
    answers.append(best_time)
    stories.append(story)

    questions.append(f"What is rating of {place}")
    answers.append(rating)
    stories.append(story)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

all_text = stories + questions + answers

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

vocab_size = len(tokenizer.word_index) + 1

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

story_seq = tokenizer.texts_to_sequences(stories)
question_seq = tokenizer.texts_to_sequences(questions)
answer_seq = tokenizer.texts_to_sequences(answers)

max_story_len = max(len(x) for x in story_seq)
max_question_len = max(len(x) for x in question_seq)

X_story = pad_sequences(story_seq, maxlen=max_story_len)
X_question = pad_sequences(question_seq, maxlen=max_question_len)

In [15]:
from tensorflow.keras.utils import to_categorical
import numpy as np

answer_tokens = [x[0] for x in answer_seq]
y = to_categorical(answer_tokens, num_classes=vocab_size)

In [16]:
from sklearn.model_selection import train_test_split

Xs_train, Xs_test, Xq_train, Xq_test, y_train, y_test = train_test_split(
    X_story, X_question, y, test_size=0.2
)

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate

input_story = Input(shape=(max_story_len,))
x1 = Embedding(vocab_size, 64)(input_story)
x1 = LSTM(32)(x1)

input_question = Input(shape=(max_question_len,))
x2 = Embedding(vocab_size, 64)(input_question)
x2 = LSTM(32)(x2)

merged = concatenate([x1, x2])

output = Dense(vocab_size, activation="softmax")(merged)

model = Model([input_story, input_question], output)

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 25, 64)    │     47,168 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 11, 64)    │     47,168 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │     12,416 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │     12,416 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 737)       │     47,905 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 167,073 (652.63 KB)

 Trainable params: 167,073 (652.63 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit([Xs_train, Xq_train], y_train,
          batch_size=32,
          epochs=100,
          validation_data=([Xs_test, Xq_test], y_test))

Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.1939 - loss: 6.4419 - val_accuracy: 0.2423 - val_loss: 4.7345
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.2468 - loss: 4.1757 - val_accuracy: 0.2423 - val_loss: 3.5601
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.2599 - loss: 3.3907 - val_accuracy: 0.2423 - val_loss: 3.4865
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.2567 - loss: 3.3255 - val_accuracy: 0.2423 - val_loss: 3.4468
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.2924 - loss: 3.3388 - val_accuracy: 0.2423 - val_loss: 3.3936
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2450 - loss: 3.1843 - val_accuracy: 0.3654 - val_loss: 3.3319
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3820 - loss: 3.0601 - val_accuracy: 0.3654 - val_loss: 3.2742
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3383 - loss: 3.1301 - val_accuracy: 

In [19]:
import numpy as np

reverse_word_index = {v:k for k,v in tokenizer.word_index.items()}

def get_story_for_place(place):

    place_data = data[data["Name"].str.lower() == place.lower()]

    if len(place_data) == 0:
        return None

    row = place_data.iloc[0]

    story = f"{row['Name']} is in {row['City']} {row['State']}. It is a {row['Type']}. Best time to visit is {row['Best Time to visit']}. Rating is {row['Google review rating']}."

    return story

In [20]:
def chatbot():

    while True:

        place = input("\nEnter place name (or type 'exit'): ")

        if place.lower() == "exit":
            print("Thank you for using Travel Bot ✈️")
            break

        story = get_story_for_place(place)

        if story is None:
            print("❌ Place not found in dataset")
            continue

        question = input("Ask your question: ")

        # vectorize
        story_seq = tokenizer.texts_to_sequences([story])
        question_seq = tokenizer.texts_to_sequences([question])

        story_pad = pad_sequences(story_seq, maxlen=max_story_len)
        question_pad = pad_sequences(question_seq, maxlen=max_question_len)

        pred = model.predict([story_pad, question_pad])

        idx = np.argmax(pred)

        if idx in reverse_word_index:
            print("🤖 Answer:", reverse_word_index[idx])
        else:
            print("🤖 Answer not found")

In [ ]:
chatbot()


Enter place name (or type 'exit'): Auroville
Ask your question: Best to visit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
🤖 Answer: morning

Enter place name (or type 'exit'): Best time to vist Auroville
❌ Place not found in dataset

Enter place name (or type 'exit'): Auroville
Ask your question: Best time to vist Auroville
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🤖 Answer: all

Enter place name (or type 'exit'): Brihadeeswarar
❌ Place not found in dataset

Enter place name (or type 'exit'): Brihadeeswarar Temple
Ask your question: Where is Brihadeeswarar Temple located
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🤖 Answer: uttarakhand
